**Curso de Inteligencia Artificial y Aprendizaje Profundo**


# Introducción al aprendizaje reforzado

## Q-Learning con redes neuronales, algoritmo DQN.

##  Autores

1. Alvaro Mauricio Montenegro Díaz, ammontenegrod@unal.edu.co
2. Daniel Mauricio Montenegro Reyes, dextronomo@gmail.com 
3. Oleg Jarma, ojarmam@unal.edu.co
4. Maria del Pilar Montenegro, pmontenegro88@gmail.com

## Contenido

* [Introducción](#Introducción)
* [Ecuación de Bellman en DQN](#Ecuación-de-Bellman-en-DQN)
* [El código](#El-código)


## Referencias

1. Adaptado de Markel Sanz, [Introducción al aprendizaje por refuerzo](https://medium.com/@markelsanz14/introducci%C3%B3n-al-aprendizaje-por-refuerzo-parte-3-q-learning-con-redes-neuronales-algoritmo-dqn-bfe02b37017f)
2. Sutton, R. S., & Barto, A. G. (2018). [Reinforcement learning: An introduction. MIT press](https://web.stanford.edu/class/psych209/Readings/SuttonBartoIPRLBook2ndEd.pdf).

## Introducción

En la lección del algoritmo Q-Learning, este  funciona muy bien cuando el entorno es simple y la función $Q(s,a)$ se puede representar como una tabla o matriz de valores. 

Pero cuando hay miles de millones de estados diferentes y cientos de acciones distintas, la tabla se vuelve enorme, y no es viable su utilización. 

Por ello, Mnih et al. inventaron el algoritmo [Deep Q-Network o DQN.](https://web.stanford.edu/class/psych209/Readings/MnihEtAlHassibis15NatureControlDeepRL.pdf)  

Este algoritmo combina el algoritmo *Q-learning* con redes neuronales profundas (Deep Neural Networks). 

Como es sabido en el campo de la IA, las redes neuronales son una fantástica manera de aproximar funciones no lineales. 

Por lo tanto, este algoritmo usa una red neuronal para aproximar la función *Q*, evitando así utilizar una tabla para representar la misma. 

En realidad, utiliza dos redes neuronales para estabilizar el proceso de aprendizaje. 

1. La primera, la *red neuronal principal* (main Neural Network), representada por los parámetros $\theta$, se utiliza para estimar los valores-Q del estado s y acción a actuales: $Q(s, a;\theta)$. 
2. La segunda, la *red neuronal objetivo* (target Neural Network), parametrizada por $\theta^{'}$, tendrá la misma arquitectura que la red principal pero se usará para aproximar los *valores-Q* del siguiente estado $s'$ y la siguiente acción $a'$. 

El aprendizaje ocurre en la red principal y no en la objetivo.

La red objetivo se congela (sus parámetros no se cambian) durante varias iteraciones (normalmente alrededor de 10000), y después los parámetros de la red principal se copian a la red objetivo, transmitiendo así el aprendizaje de una a otra, haciendo que las estimaciones calculadas por la red objetivo sean más precisas.

## Ecuación de Bellman en DQN

La ecuación de Bellman tiene esta forma ahora. 

$$
\large
Q(s,a; \theta) = r + \lambda \max_{a^{'}}  Q(s^{'},a^{'}; \theta^{'}).
$$


Para poder entrenar una red neuronal, necesitamos una función de pérdida o coste (loss or cost function), la cual definimos como el cuadrado de la diferencia entre ambos lados de la ecuación de Bellman:

$$
\large
L(\theta)= \mathbb{E}[  r + \lambda \max_{a^{'}}  Q(s^{'},a^{'}; \theta^{'})- Q(s,a; \theta)]^2.
$$


Esta será la función que minimizaremos usando el algoritmo de descenso de gradiente (gradient descent), el cuál se ejecuta automáticamente si usamos una librería de diferenciación automática con redes neuronales, como TensorFlow.

Aquí tenemos el entorno (environment) conocido como CartPole. Se ha utilizado la librería *OpenAI Gym* para visualizar y ejecutar este entorno. En este entorno, el objetivo es mover el carro hacia la derecha y la izquierda, con el objetivo de equilibrar el palo. Y si el palo se tuerce más de 15 grados respecto al eje vertical, el episodio terminará y volveremos a empezar.

<figure>
<center>
<img src="../Imagenes/Ejemplo_polea.gif" width="400" height="300" align="center"/>
    <figcaption>
<p style="text-align:center">Ejemplo de la polea. Original en Open AI</p>
</figcaption>
</figure>

Fuente: [Introducción al aprendizaje por refuerzo](https://medium.com/@markelsanz14/introducci%C3%B3n-al-aprendizaje-por-refuerzo-parte-3-q-learning-con-redes-neuronales-algoritmo-dqn-bfe02b37017f)

## El código

Para implementar el algoritmo DQN, empezaremos creando las dos redes neuronales, la principal (main_nn) y la objetivo (target_nn). Ésta última será una copia de la principal, pero con sus propios pesos. 

También necesitaremos un optimizador (optimizer) y una función de pérdida (loss function).

In [ ]:
class DQN(tf.keras.Model):
  """Dense neural network class."""
  def __init__(self):
    super(DQN, self).__init__()
    self.dense1 = tf.keras.layers.Dense(32, activation="relu")
    self.dense2 = tf.keras.layers.Dense(32, activation="relu")
    self.dense3 = tf.keras.layers.Dense(num_actions, dtype=tf.float32) # No activation
    
  def call(self, x):
    """Forward pass."""
    x = self.dense1(x)
    x = self.dense2(x)
    return self.dense3(x)

main_nn = DQN()
target_nn = DQN()

optimizer = tf.keras.optimizers.Adam(1e-4)
mse = tf.keras.losses.MeanSquaredError()

Ahora crearemos el buffer donde guardaremos la experiencia recogida para usarla después y entrenar la red neuronal.

Usaremos *deque* — Cola doblemente terminada.

Una cola doblemente terminada, o *deque*, admite agregar y eliminar elementos de cualquier extremo de la cola. Las pilas y colas más comunes son formas degeneradas de deques, donde las entradas y salidas están restringida a un solo extremo.

In [ ]:
class ReplayBuffer(object):
  """Experience replay buffer that samples uniformly."""
  def __init__(self, size):
    self.buffer = deque(maxlen=size)

  def add(self, state, action, reward, next_state, done):
    self.buffer.append((state, action, reward, next_state, done))

  def __len__(self):
    return len(self.buffer)

  def sample(self, num_samples):
    states, actions, rewards, next_states, dones = [], [], [], [], []
    idx = np.random.choice(len(self.buffer), num_samples)
    for i in idx:
      elem = self.buffer[i]
      state, action, reward, next_state, done = elem
      states.append(np.array(state, copy=False))
      actions.append(np.array(action, copy=False))
      rewards.append(reward)
      next_states.append(np.array(next_state, copy=False))
      dones.append(done)
    states = np.array(states)
    actions = np.array(actions)
    rewards = np.array(rewards, dtype=np.float32)
    next_states = np.array(next_states)
    dones = np.array(dones, dtype=np.float32)
    return states, actions, rewards, next_states, dones

También crearemos una función auxiliar para ejecutar la política ε-voraz, y otra para entrenar la red neuronal principal usando los datos guardados en el buffer.

In [ ]:
def select_epsilon_greedy_action(state, epsilon):
  """Take random action with probability epsilon, else take best action."""
  result = tf.random.uniform((1,))
  if result < epsilon:
    return env.action_space.sample() # Random action (left or right).
  else:
    return tf.argmax(main_nn(state)[0]).numpy() # Greedy action for state.

@tf.function
def train_step(states, actions, rewards, next_states, dones):
  """Perform a training iteration on a batch of data sampled from the experience
  replay buffer."""
  # Calculate targets.
  next_qs = target_nn(next_states)
  max_next_qs = tf.reduce_max(next_qs, axis=-1)
  target = rewards + (1. - dones) * discount * max_next_qs
  with tf.GradientTape() as tape:
    qs = main_nn(states)
    action_masks = tf.one_hot(actions, num_actions)
    masked_qs = tf.reduce_sum(action_masks * qs, axis=-1)
    loss = mse(target, masked_qs)
  grads = tape.gradient(loss, main_nn.trainable_variables)
  optimizer.apply_gradients(zip(grads, main_nn.trainable_variables))
  return loss

Tras esto, definiremos los hiperparámetros y empezaremos a entrenar el algoritmo. Para ello, primero usaremos la **política ε-voraz** para jugar al juego y recoger experiencia para poder aprender de esos datos. 

Después de terminar un episodio jugado, llamaremos a la función que entrena la red neuronal. 

Cada 2000 pasos de entrenamiento, copiaremos los pesos de la red neuronal principal a la red neuronal objetivo. También reduciremos el valor de **epsilon (ε)**, para empezar con un valor de exploración alto y bajarlo poco a poco. 

Así, veremos cómo el algoritmo empieza a aprender a jugar al juego y la recompensa obtenida jugando al juego irá mejorando poco a poco.

In [ ]:
# Hyperparameters.
num_episodes = 1000
epsilon = 1.0
batch_size = 32
discount = 0.99
buffer = ReplayBuffer(100000)
cur_frame = 0

# Start training. Play game once and then train with a batch.
last_100_ep_rewards = []
for episode in range(num_episodes+1):
  state = env.reset()
  ep_reward, done = 0, False
  while not done:
    state_in = tf.expand_dims(state, axis=0)
    action = select_epsilon_greedy_action(state_in, epsilon)
    next_state, reward, done, info = env.step(action)
    ep_reward += reward
    # Save to experience replay.
    buffer.add(state, action, reward, next_state, done)
    state = next_state
    cur_frame += 1
    # Copy main_nn weights to target_nn.
    if cur_frame % 2000 == 0:
      target_nn.set_weights(main_nn.get_weights())

    # Train neural network.
    if len(buffer) >= batch_size:
      states, actions, rewards, next_states, dones = buffer.sample(batch_size)
      loss = train_step(states, actions, rewards, next_states, dones)
  
  if episode < 950:
    epsilon -= 0.001

  if len(last_100_ep_rewards) == 100:
    last_100_ep_rewards = last_100_ep_rewards[1:]
  last_100_ep_rewards.append(ep_reward)
    
  if episode % 50 == 0:
    print(f'Episode {episode}/{num_episodes}. Epsilon: {epsilon:.3f}. '
          f'Reward in last 100 episodes: {np.mean(last_100_ep_rewards):.3f}')
env.close()

Ahora que el agente ha aprendido a maximizar la recompensa para el entorno CartPole, haremos que el agente interactúe con el entorno una vez más, y visualizamos el resultado, viendo como ahora es capaz de mantener el palo equilibrado durante 200 frames.


<figure>
<center>
<img src="../Imagenes/Ejemplo_polea_entrenado.gif" width="400" height="300" align="center"/>
    <figcaption>
<p style="text-align:center">Ejemplo de la polea despues de entrenar al agente. Original en Open AI</p>
</figcaption>
</figure>

Fuente: [Introducción al aprendizaje por refuerzo](https://medium.com/@markelsanz14/introducci%C3%B3n-al-aprendizaje-por-refuerzo-parte-3-q-learning-con-redes-neuronales-algoritmo-dqn-bfe02b37017f)